In [28]:
!pip install git+https://github.com/konbraphat51/line_minimum_interception.git

  Cloning https://github.com/konbraphat51/line_minimum_interception.git to /private/var/folders/j_/6b6v3sh15494_n31hkc0sh6w0000gn/T/pip-req-build-mvlrox2k
  Running command git clone --filter=blob:none --quiet https://github.com/konbraphat51/line_minimum_interception.git /private/var/folders/j_/6b6v3sh15494_n31hkc0sh6w0000gn/T/pip-req-build-mvlrox2k
  Resolved https://github.com/konbraphat51/line_minimum_interception.git to commit 805b21443d47cf4c7b9d0c7741255d9a8ff93ada
  Preparing metadata (setup.py) ... done

[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip


In [29]:
import numpy as np
import math
from scipy.spatial.transform import Rotation as R

In [30]:
def rotate_vector(vec, theta, phi):
    """
    Rotate a 3D vector first around the Z-axis by theta, then rotate in the plane
    containing the Z-axis and the vector by phi.

    Parameters:
    v (array-like): The 3D vector to rotate.
    theta (float): The angle in radians to rotate around the Z-axis.
    phi (float): The angle in radians to rotate in the Z-vector plane.

    Returns:
    np.array: The rotated vector.
    """

    # Step 1: Z軸中心にtheta回転
    rotation_z = R.from_euler('z', theta, degrees=False)
    v_rotated = rotation_z.apply(vec)

    # Step 2: 水平線を中心にphi回転
    z_axis = np.array([0, 0, 1])
    rotation_axis = np.cross(z_axis, v_rotated)
    if np.linalg.norm(rotation_axis) != 0:  # Check if the rotation axis is valid
        rotation_axis /= np.linalg.norm(rotation_axis)  # Normalize the rotation axis
        rotation_plane = R.from_rotvec(phi * rotation_axis)
        v_rotated = rotation_plane.apply(v_rotated)

    return v_rotated

In [31]:
def adjust_vector(pos_vec, dir_vec, bb_size, image_size, horizontal_fov):
    # バウンディングボックスのサイズを取得
    
    
    # YOLOのBBの中心座標分だけ方向ベクトルを補正する関数
    bb_center = [(axis[0] + axis[1] / 2) for axis in bb_size]
    theta = (image_size[0] / 2 - bb_center[0]) / (image_size[0] / 2) * horizontal_fov / 2 * math.pi / 180 #水平方向の回転角 左向き正
    phi = -(image_size[1] / 2 - bb_center[1]) / (image_size[0] / 2) * horizontal_fov / 2 * math.pi / 180 #鉛直方向の回転角 上向き正
    
    return rotate_vector(dir_vec, theta, phi)

In [32]:
class input_set:
    def __init__(self, pos_vec, dir_vec, bb_size, image_size):
        self.pos_vec = pos_vec
        self.dir_vec = dir_vec
        self.bb_size = bb_size
        self.image_size = image_size

In [43]:
# input
horizontal_fov = 39.6

inputs = []
inputs.append(input_set([6, 0, 0], [-1, 0, 0], [[616, 311], [964, 618]], [1234, 684]))
inputs.append(input_set([0, 6, 0], [0, -1, 0], [[614, 268], [1121, 489]], [1234, 684]))

In [44]:
from line_minimum_intersection import Vector, Line, calculate_minimum_movements

In [45]:
lines = []

for raw_input in inputs:
    print(f"original vector: {raw_input.dir_vec}")
    adj_dir_vec = adjust_vector(raw_input.pos_vec, raw_input.dir_vec, raw_input.bb_size, raw_input.image_size, horizontal_fov)
    print(f"adjusted vector: {adj_dir_vec}")
    pos_vec = raw_input.pos_vec
    lines.append(Line(Vector(pos_vec[0], pos_vec[1], pos_vec[2]), Vector(pos_vec[0]+adj_dir_vec[0], pos_vec[1]+adj_dir_vec[1], pos_vec[2]+adj_dir_vec[2])))

movement_vectors = calculate_minimum_movements(lines)

print("intersection point:")
print(movement_vectors["intersection_point"])

# print("movement vectors:")
# for vector in movement_vectors["movement_vectors"]:
#    print(vector)

original vector: [-1, 0, 0]
adjusted vector: [-0.86385669  0.07493997 -0.49813213]
original vector: [0, -1, 0]
adjusted vector: [-0.06158736 -0.83788149 -0.54236668]
intersection point:
(0.06950191621129605, -0.08319926028140713, 0.6071272729094681)
